# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App 

This notebook accepts any user-supplied image as input. If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

*Note: If you are using a Windows machine, you are encouraged to use [7zip](http://www.7-zip.org/) to extract the folder.*

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [ ]:
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("lfw/*/*"))
dog_files = np.array(glob("dog_images/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[0])

# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2) # add bounding box to color image
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Human Face Detector

We use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
def face_detector(img_path):
    '''
    Returns:
       "True" if face is detected in image stored at img_path
    '''
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### (IMPLEMENTATION) Assess the Human Face Detector

__Question 1:__ Use the code cell below to test the performance of the `face_detector` function.  
- What percentage of the first 100 images in `human_files` have a detected human face?  
- What percentage of the first 100 images in `dog_files` have a detected human face? 

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

__Answer:__ 
The exact percentages are plotted below. We observe that the face detector is able to classify most human faces correctly, but it also mistakes about a fifth of the dogs for humans.

In [ ]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]


def picsWithHumans(filelist):
    isHumanList = [1 if face_detector(file) else 0 for file in filelist]
    return np.sum(isHumanList)


ratio_human_classified = picsWithHumans(human_files_short) / len(human_files_short)
ratio_dogs_classified = 1 - (picsWithHumans(dog_files_short) / len(dog_files_short))

print(f"Correctly classified pics of humans: {ratio_human_classified * 100} percent")
print(f"Correctly classified pics of dogs: {ratio_dogs_classified * 100} percent")

We suggest the face detector from OpenCV as a potential way to detect human images in your algorithm.

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

The following function VGG_predict accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  he output is an integer between 0 and 999, inclusive.

Tensors are pre-processed for pre-trained models following the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
        
in_transform_train = transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomRotation(10),
                        transforms.RandomResizedCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

in_transform_test = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])


def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    ## Load and pre-process an image from the given img_path 
    image = Image.open(img_path).convert('RGB')

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = in_transform_test(image)[:3,:,:].unsqueeze(0)
    
    # move to gpu if available
    content = image.to("cuda" if use_cuda else "cpu")
    
    # set to evaluation mode
    VGG16.eval()
    
    # predict
    pred_layer = VGG16(content)
    _, prediction = torch.max(pred_layer, 1)
    
    return prediction

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

This idea is used to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
def dog_detector(img_path):
    '''
    Returns:
        "True" if a dog is detected in the image stored at img_path
    '''
    
    pred_class = VGG16_predict(img_path)  
    
    return (151 <= pred_class <= 268)

### (IMPLEMENTATION) Assess the Dog Detector

__Question 2:__ Use the code cell below to test the performance of your `dog_detector` function.  
- What percentage of the images in `human_files_short` have a detected dog?  
- What percentage of the images in `dog_files_short` have a detected dog?

__Answer:__ The exact percentages are given below. The dog detector is very accurate on the given data set.


In [ ]:
def picsWithHumansVGG(filelist):
    isHumanList = [0 if dog_detector(file) else 1 for file in filelist]    
    return np.sum(isHumanList)


ratio_human_classified = picsWithHumansVGG(human_files_short) / len(human_files_short)
ratio_dogs_classified = 1 - (picsWithHumansVGG(dog_files_short) / len(dog_files_short))

print(f"Correctly classified pics of humans: {ratio_human_classified * 100} percent")
print(f"Correctly classified pics of dogs: {ratio_dogs_classified * 100} percent")

We suggest VGG-16 as a potential network to detect dog images in your algorithm.

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, we will create a CNN that classifies dog breeds. We are first creating our CNN _from scratch_ (so, we are not using transfer learning _yet_!), and try to attain a test accuracy of at least 10%.  In Step 4 of this notebook, we will use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging. To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="images/Brittany_02625.jpg" width="100"> | <img src="images/Welsh_springer_spaniel_08203.jpg" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="images/Curly-coated_retriever_03896.jpg" width="200"> | <img src="images/American_water_spaniel_00648.jpg" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="images/Labrador_retriever_06457.jpg" width="150"> | <img src="images/Labrador_retriever_06455.jpg" width="240"> | <img src="images/Labrador_retriever_06449.jpg" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%. 

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

The code cell below is used to provide three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dog_images/train`, `dog_images/valid`, and `dog_images/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
import os
from torchvision import datasets

test_data = datasets.ImageFolder('dog_images/test', transform=in_transform_test)
train_data = datasets.ImageFolder('dog_images/train', transform=in_transform_train)
val_data = datasets.ImageFolder('dog_images/valid', transform=in_transform_test)

print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))
print('Num validation images: ', len(val_data))

# define dataloader parameter
batch_size = 20

# prepare data loaders
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**: The transformations are written in step 2. For testing and validation I decided to stay very close to the original images. To be compatible with the network input the images are resized to 256x256 and the center cropped to 224x224. The normalization parameters are taken from the ImageNet-dataset, since our dog classifier will use the pretrained network in the end (and therefor should be normalized by the same mean and standard deviation) and it should also work fine for our customly designed CNN since the parameters are calculated from a very high number of 'real' images within the ImageNet sdataset.
For training we randomly flip and rotate the images by up to 10 degrees and then randomly resize and crop to 224x224. This is done to generate more individual training data.

### (IMPLEMENTATION) Model Architecture

Creating a CNN to classify dog breed.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 64, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 133)
        
        self.dropout = nn.Dropout(0.2)                 
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))        
        x = x.view(-1, 64 * 14 * 14)        
        x = self.dropout(x)        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
                 
        return x

    
# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

__Question 4:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  

__Answer:__ I added several convolutional layers to extract higher level features from the images. Each of the steps is followed by max pooling layer with a kernel size and stride of 2. This halves the width and height of the images in each step, thus downscales the images within the layers and reduces the number of parameters within the network.
The classification layers are given by fully connected layers. First we map all the remaining pixels of the 4th max pooling layer to 512 nodes and then we map these 512 nodes to our final 133 classification nodes.

To avoid overfitting I also added a dropout layer and to not get in trouble with vanishing gradients or slow learning, I decided to use ReLu activation functions.

### (IMPLEMENTATION) Specify Loss Function and Optimizer

We are specifying the [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  The chosen loss function are saved as `criterion_scratch`, and the optimizer as `optimizer_scratch`.

In [ ]:
import torch.optim as optim

criterion_scratch = nn.CrossEntropyLoss()

optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01)

loaders_scratch = {"train": train_loader,
                   "test": test_loader,
                   "valid": val_loader}

### (IMPLEMENTATION) Train and Validate the Model

The model will be trained and validated in the code cell below.  [The final model parameters will be saved](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [ ]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs + 1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            
            # calculate the batch loss
            loss = criterion(output, target)
            
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            
            # perform a single optimization step (parameter update)
            optimizer.step()
            
            # update training loss
            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)

        ######################    
        # validate the model #
        ######################
        model.eval()
        
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            
            # calculate the batch loss
            loss = criterion(output, target)
            
            # update average validation loss 
            valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss)

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print(f"Validation loss decreased from {valid_loss_min} to {valid_loss}. Saving model.")
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    return model


# train the model
model_scratch = train(50, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

### (IMPLEMENTATION) Test the Model

Testing the model on the test dataset of dog images. The code cell below is used to calculate and print the test loss and accuracy.

In [ ]:
def test(loaders, model, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
            
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the loss
        loss = criterion(output, target)
        
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step4'></a>
## Step 4: Creating a CNN to Classify Dog Breeds (using Transfer Learning)

We will now use transfer learning to create a CNN that can identify dog breed from images and we want to reach 60% accuracy on the test set or more.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

The code cell below is used to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

We are using the same data loaders as previously.

In [ ]:
loaders_transfer = {"train": train_loader,
                    "test": test_loader,
                    "valid": val_loader}

### (IMPLEMENTATION) Model Architecture

Using transfer learning to create a CNN to classify dog breed. The initialized model is saved as the variable `model_transfer`.

In [ ]:
import torchvision.models as models
import torch.nn as nn

## Specify model architecture
model_transfer = models.vgg16(pretrained=True)

# Freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False

# Set output layer to reflect 133 classes.
model_transfer.classifier[6] = nn.Linear(4096, 133)

if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ We are loading the pretrained VGG16 and freezing all feature layers. Since our dataset is reasonably big, I decided to use the same classifier architecture. Only the last layers needs to be changed to reflect that we are only differentiating between 133 classes. During training, we will only update the parameters of the classification layers.


### (IMPLEMENTATION) Specify Loss Function and Optimizer

In the next code cell we are specifying the [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html). The chosen loss function is saved as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion_transfer = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr=0.001)

### (IMPLEMENTATION) Train and Validate the Model

We are training and validating our model in the code cell below.  [The final model parameters are saved](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
model_transfer = train(5, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

We are applying our model to predict the images of the test dataset. Use the code cell below also calculates and prints the test loss and accuracy.

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

The following function takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by our model.  

In [ ]:
# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in train_data.classes]

def predict_breed_transfer(img_path):
    '''
    load the image and return the predicted breed    
    '''
    
    image = Image.open(img_path).convert('RGB')

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = in_transform_test(image)[:3,:,:].unsqueeze(0)
    content = image.to("cuda" if use_cuda else "cpu")
    
    model_transfer.eval()
    pred_layer = model_transfer(content)
    _, prediction = torch.max(pred_layer, 1)
    
    return class_names[prediction]

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Here we provide an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

### (IMPLEMENTATION) Write your Algorithm

In [ ]:
def plot_img(img_path):    
    # load color (BGR) image
    img = cv2.imread(img_path)

    # convert BGR image to RGB for plotting
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # display the image, along with bounding box
    plt.imshow(cv_rgb)
    plt.show()
    

def run_app(img_path):
    ## handle cases for a human face, dog, and neither    
    if dog_detector(img_path):
        breed = predict_breed_transfer(img_path)
        postfix = "n" if breed[0].lower() in {"a", "e", "i", "o", "u"} else ""
        print(f"This dog looks a lot like a{postfix} {breed}")
    elif face_detector(img_path):
        breed = predict_breed_transfer(img_path)
        postfix = "n" if breed[0].lower() in {"a", "e", "i", "o", "u"} else ""
        print(f"Hello, human! You look a lot like a{postfix} {breed}.")
    else:
        print("ERROR: The given image neither shows a dog nor a human.") 
        
    plot_img(img_path)
    print(f"Image path: {img_path}")    
    print("\n")
    

---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, we will take our new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ Having a look at many images (and also considering the accuracies of the dog and face detector given above), our algorithm sometimes fails to see that there is a dog or human in an image. Also the dog with glasses (have a look at the outputs below) is mistakenly classified as a bulldog. I am not an expert on dog breeds, but I do not think this is right.

Three suggestions to improve the algorithm:
- To detect human faces it might be more accurate to also train a CNN (or use a pretrained network) than using a Haar feature-based cascade classifier.
- The training images generator could also randomly occlude parts of the images to be able to find more features to detect a dog breed (and thus also be able to classify a dog when it wears glasses e.g.)
- The algorithm will give strange outputs when we use an image with multiple dogs, or dog and human as an input. We could try to at least give a warning message if this is the case.

In [ ]:
import random

for file in np.hstack((random.sample(list(human_files), 3), random.sample(list(dog_files), 3))):
    run_app(file)

In [ ]:
run_app("own_images/dog_glasses.jpg")

In [ ]:
run_app("own_images/moi.png")